<a href="https://colab.research.google.com/github/BNIA/VitalSigns/blob/main/Mdprop_closecrawl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MDProp Intake and Operations

> This notebook uses Info-USA data to generate a portion of BNIA's Vital Signs report.

#### __Columns Used__

- ✅ 14x - __crehab__ - (MdProp, Permits) Percentage of properties with rehabilitation permits exceeding $5k

- ✅ 14x - __comprop__ - (MdProp, Permits) Percentage of properties with rehabilitation permits exceeding $5k

- ✅ 36 - __resrehab__ - (MdProp, Permits) Percentage of properties with rehabilitation permits exceeding $5k

- ✅ 41 - __demper__ - (MdProp, Permits) number of demolition permits per 1000 residential properties

- ✅ 42 - __constper__ - (MdProp, Permits) Number of of new constructio permits per 1000 residential properties

#### __Datasets Used__

 
- ✅ foreclosures.foreclosures_201X __(33-fore-> 2018/ 2019 Close-Crawler)__

- ❌ housing.vacants_201X __(34-vacant -> datenotice, dateabate, datecancle)__

- ✔️ housing.permits_201X __(36-resrehab, 41-demper, 42-constper -> field22, casetype, propuse, existingus, cost)__

- ✔️ housing.mdprop_201X 
- - 18 can use the indicators from 2017. 
- - [totalres](https://bniajfi.org/indicators/Housing%20And%20Community%20Development/totalres) __( 33-fore, 34-vacant, 35-vio, 36-resrehab, 40-taxlien, 41-demper, 42-constper, __ ownroc __)__ 


This colab and more can be found at https://github.com/BNIA/colabs.


## About this Tutorial: 

### Whats Inside?

#### __The Tutorial__

This notebook was made to create Vital Signs Indicators from an Info-USA geographic dataset.

This lab is split into two sections. 
- The first part of this lab, a _guided walkthrough_, provides ample documentation so you understand how to perform basic geographic data operations.
- The second, more _advanced_ part of this notebook provides a single python function that handles everything covered in this lab (and more). 

#### __Objectives__

- Reading in data (points/ geoms)
-- Convert lat/lng columns to point coordinates
-- Geocoding address to coordinates
-- Changing coordinate reference systems
-- Connecting to PostGisDB's
- Basic Operations
- Saving shape data
- Get Polygon Centroids
- Working with Points and Polygons
-- Map Points and Polygons
-- Get Points in Polygons
-- Create Choropleths
-- Create Heatmaps (KDE?)

# Guided Walkthrough

## SETUP:

### Import Modules

In [ ]:
%%capture
! pip install -U -q PyDrive
! pip install geopy
! pip install geopandas
! pip install geoplot

In [ ]:
!apt install libspatialindex-dev
!pip install rtree

In [ ]:
%%capture
!pip install dataplay

In [ ]:
%%capture 
# These imports will handle everything
import os
import sys
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import psycopg2
import pyproj
from pyproj import Proj, transform
# conda install -c conda-forge proj4
from shapely.geometry import Point
from shapely import wkb
from shapely.wkt import loads
# https://pypi.org/project/geopy/
from geopy.geocoders import Nominatim

# In case file is KML, enable support
import fiona
fiona.drvsupport.supported_drivers['kml'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'

In [ ]:
from IPython.display import clear_output
clear_output(wait=True)

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

### Configure Enviornment

In [ ]:
# This will just beautify the output

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# pd.set_option('display.expand_frame_repr', False)
# pd.set_option('display.precision', 2)
# pd.reset_option('max_colwidth')
pd.set_option('max_colwidth', 20)
# pd.reset_option('max_colwidth')

## CSA and Baltimore

In [ ]:
#collapse_output
#collapse_input
csa = "https://services1.arcgis.com/mVFRs7NF4iFitgbY/ArcGIS/rest/services/Tpop/FeatureServer/1/query?where=1%3D1&outFields=*&returnGeometry=true&f=pgeojson"
csa = gpd.read_file(csa);
csa.head(1)

In [ ]:
url2 = "https://services1.arcgis.com/mVFRs7NF4iFitgbY/ArcGIS/rest/services/Tpop/FeatureServer/0/query?where=1%3D1&outFields=*&returnGeometry=true&f=pgeojson"
csa2 = gpd.read_file(url2);
csa2['CSA2010'] = csa2['City_1'] 
csa2['OBJECTID'] = 56 
csa2 = csa2.drop(columns=['City_1'])
csa2.head()

In [ ]:
# csa = pd.concat([csa2, csa], ignore_index=True)
csa = csa.append(csa2).reset_index(drop=True)

Actually...

In [ ]:
totalres = pd.read_csv('37-totalres-19.csv')

Append Baltimore City to the top of the CSA list. You use concat and not append to do this. We put it on the top of the df because when performing the ponp it returns only the last matching columns CSA Label. 

In [ ]:
totalres.head()
totalres.tail()

# Permits

## Import

In [ ]:
permits = gpd.read_file("Permits_2019_CSACity.shp");
permits.head()

In [ ]:
permits.columns
permits.crs
permits.head(5)

In [ ]:
# Convert to EPSG:4326
permits = permits.to_crs(epsg=4326)
permits.crs

In [ ]:
# Convert Geom to Coords
permits['x'] = permits.geometry.x
permits['y'] = permits.geometry.y
permits.head(5)

In [ ]:
permits = permits[ permits.geometry.y > 38 ]

In [ ]:
# Reference: All Points
base = csa.plot(color='white', edgecolor='black')
permits.plot(ax=base, marker='o', color='green', markersize=5);

In [ ]:
permits.columns

In [ ]:
csa.tail()

In [ ]:
from dataplay import geoms
# Get CSA Labels for all Points.
# permitsCsa = geoms.getPolygonOnPoints(permits, csa, 'geometry', 'geometry', 'CSA2010' )
permitsCsa = geoms.workWithGeometryData(method='ponp', df=permits, polys=csa, ptsCoordCol='geometry', polygonsCoordCol='geometry', polyColorCol=False, polygonsLabel='CSA2010', pntsClr='red', polysClr='white')
# permitsCsa = permitsCsa.drop('geometry',axis=1)

In [ ]:
permitsCsa.head(10)

## Processing

All

In [ ]:
permitsAll = permits

In [ ]:
# Reference: All Points
base = csa.plot(color='white', edgecolor='black')
permitsAll.plot(ax=base, marker='o', color='green', markersize=5);

In [ ]:
permits = permitsAll

In [ ]:
# y < 0
permitsLessThanZero = permits[ permits.geometry.y < 0 ]
print('Y<0: ', permitsLessThanZero.size, '\n')

In [ ]:
# y > 0
permitsGreaterThanZero = permits[ permits.geometry.y > 0 ]
print('Y>0: ', permitsGreaterThanZero.size, '\n')
permitsGreaterThanZero.plot();

In [ ]:
# 0 < y < 38
permitsOver38 = permits[ permits.geometry.y < 38 ]
permitsOver38 = permitsOver38[ permitsOver38.geometry.y > 0 ]
print('0 < y < 38: ', permitsOver38.size, '\n') 

In [ ]:
# y > 38
permitsUnder38 = permits[ permits.geometry.y > 38 ]
print('Y>38 Less than Zero: ', permitsUnder38.size, '\n')
permitsUnder38.plot();

# Create Indicators

### Load Data

In [ ]:
totalres = pd.read_csv('37-totalres-19.csv')

In [ ]:
# https://stackoverflow.com/questions/29314033/drop-rows-containing-empty-cells-from-a-pandas-dataframe
year = '19'
permitsCsa = gpd.read_file("Permits_20"+year+"_CSACity.shp", geometry='geometry');
# print(permitsCsa.crs)
# permits = permits.to_crs(epsg=4326)
permitsCsa['CSA2010'] = permitsCsa['CSA']
permitsCsa.head()

permitsCsa['bcitycount'] = 1
permitsCsa['csacount'] = 1

print( permitsCsa[permitsCsa['BaltCity'].isnull()].CSA.unique() ) 

# replace empty
permitsCsa['BaltCity'].replace('', np.nan, inplace=True)
# drop null
permitsCsa.dropna(subset=['BaltCity'], inplace=True)

In [ ]:
permitsCsa.typework.unique()

In [ ]:
permitsCsa[permitsCsa['BaltCity'].isnull()].CSA.unique()

In [ ]:
permitsCsa.BaltCity.value_counts()

### Comprop 141 - (MdProp)

----/*<comprop>* Indicator Number 141/
		with tbl AS (
			select (sum(
						case 
						when (lu like 'C' OR lu LIKE 'EC' OR lu LIKE 'I')
						then 1
						else 0
						end)::numeric 
				) as result, csa
				from vital_signs.match_csas_and_bc_by_geom('housing.mdprop_2017', 'gid', 'the_geom') a
				left join housing.mdprop_2017 b on a.gid = b.gid
				group by csa, the_pop
			)
			update vital_signs.data
			set comprop = result from tbl where data.csa = tbl.csa and data_year = '2016';		



sum( case when (lu like 'C' OR lu LIKE 'EC' OR lu LIKE 'I')

In [ ]:
mdprop = gpd.read_file('MDPropertyView_2021_CSACity.shp')
pd.set_option('display.max_columns', None)
mdprop.head(1)

In [ ]:
comprop = mdprop.copy()
comprop['CSA'] = comprop.CSA.fillna('Baltimore City')
comprop['comprop19'] = 1
# mdprop = csa[['CSA','comprop19']]
comprop = comprop[ comprop['LU'].isin( ['C','EC','I'] ) ]
comprop = comprop.groupby('CSA').sum(numeric_only=True) 
# Make sure ALL csas and BaltimoreCity are included. among other things
comprop = csa[ ['CSA2010','tpop10'] ].merge( comprop, left_on='CSA2010', right_on='CSA', how='outer' ) 
# Update the baltimore CSA.
comprop.at[55,'comprop19'] = comprop['comprop19'].sum() 
comprop = comprop[['comprop19', 'CSA2010']]
comprop.head(58)

# Create the Indicator
comprop.to_csv('141-comprop19-19.csv', index=False) 

### Crehab 142 - (Permits, MdProp)


--/* <crehab_16> * Indicator Number 142/
		with numerator AS (
			select sum(
			case 
				when (
					exis LIKE any (ARRAY['COM','IND','BUS','AIR','ANIM','BAR','BEAU','DELI','FAC','ASM','ALV%','DOTH','DWC','EDU','FOOD','HCF','HIH','HOS','MIXC','INS','MER','LIB','MNTL','MOB','PUB','STO','UT','VAC','VAL','DFAM'])
					AND 
					prop LIKE any (ARRAY['COM','IND','BUS','AIR','ANIM','BAR','BEAU','DELI','FAC','ASM','ALV%','DOTH','DWC','EDU','FOOD','HCF','HIH','HOS','MIXC','INS','MER','LIB','MNTL','MOB','PUB','STO','UT','DFAM'])
					AND 
					type1 = ANY (ARRAY['AA','ALT','ADD','NEW'])
					and costts >=5000
			) 
			then 1
			else 0
			end)::numeric as result, csa
			from vital_signs.match_csas_and_bc_by_geom('housing.permits_2016', 'gid', 'the_geom') a
			left join housing.permits_2016 b on a.gid = b.gid
			group by csa
			),
			denominator AS (
				select (sum(
					case 
					when (lu like 'C' OR lu LIKE 'EC' OR lu LIKE 'I')
					then 1
					else 0
					end)::numeric 
				) as result, csa
				from vital_signs.match_csas_and_bc_by_geom('housing.mdprop_2017', 'gid', 'the_geom') a
				left join housing.mdprop_2017 b on a.gid = b.gid
				group by csa
			),
			tbl AS (
					select denominator.csa,(numerator.result / denominator.result)*(100::numeric) as result 
					from numerator left join denominator on numerator.csa = denominator.csa
				)
			update vital_signs.data
			set crehab = result from tbl where data.csa = tbl.csa and data_year = '2016';

The Above query is bad because we shouldnt filter for new in the type1 column.

Also. NO FILTERING ON PROP!

In [ ]:
permitsCsa.head(1)

In [ ]:
# '2-%','3-%','4-%','5-%','6-%','7-%', 'COM','IND','BUS','AIR','ANIM','BAR','BEAU','DELI','FAC','ASM','ALV%','DOTH','DWC','EDU','FOOD','HCF','HIH','HOS','MIXC','INS','MER','LIB','MNTL','MOB','PUB','STO','UT','VAC','VAL','DFAM'
# crehab.loc[crehab['existingus'].str.contains('2-|3-|4-|5-|6-|7-|COM|IND|BUS|AIR|ANIM|BAR|BEAU|DELI|FAC|ASM|ALV|DOTH|DWC|EDU|FOOD|HCF|HIH|HOS|MIXC|INS|MER|LIB|MNTL|MOB|PUB|STO|UT|VAC|VAL|DFAM') == True]

In [ ]:
crehab.shape[0]

In [ ]:
# get the permits file
crehab = permitsCsa.copy() 
crehab['CSA'] = crehab.CSA.fillna('Baltimore City')
crehab['crehab19'] = 1 

# Our Column to Sum on
crehab['crehab19'] = 1
# Filter 1

print('No Filter:', crehab.shape[0])
print('Filter Cost:', crehab[crehab['cost'] >=5000].cost.shape[0])
print('Filter ExistingUse:', crehab.loc[crehab['existingus'].str.contains('2-|3-|4-|5-|6-|7-|COM|IND|BUS|AIR|ANIM|BAR|BEAU|DELI|FAC|ASM|ALV|DOTH|DWC|EDU|FOOD|HCF|HIH|HOS|MIXC|INS|MER|LIB|MNTL|MOB|PUB|STO|UT|VAC|VAL|DFAM') == True].shape[0])
# print('Filter Propuse:', crehab.loc[ crehab['propuse'].str.contains('COM|IND|BUS|AIR|ANIM|BAR|BEAU|DELI|FAC|ASM|ALV|DOTH|DWC|EDU|FOOD|HCF|HIH|HOS|MIXC|INS|MER|LIB|MNTL|MOB|PUB|STO|UT|DFAM') == True].shape[0])
print('Filter typework:', crehab[  crehab['typework'].isin( ['AA','ALT','ADD'] ) ].shape[0])

crehab.loc[ crehab['propuse'].str.contains('COM|IND|BUS|AIR|ANIM|BAR|BEAU|DELI|FAC|ASM|ALV|DOTH|DWC|EDU|FOOD|HCF|HIH|HOS|MIXC|INS|MER|LIB|MNTL|MOB|PUB|STO|UT|DFAM') == True].propuse.unique()

crehab = crehab.loc[crehab['existingus'].str.contains('2-|3-|4-|5-|6-|7-|COM|IND|BUS|AIR|ANIM|BAR|BEAU|DELI|FAC|ASM|ALV|DOTH|DWC|EDU|FOOD|HCF|HIH|HOS|MIXC|INS|MER|LIB|MNTL|MOB|PUB|STO|UT|VAC|VAL|DFAM') == True]
# crehab = crehab.loc[ crehab['propuse'].str.contains('COM|IND|BUS|AIR|ANIM|BAR|BEAU|DELI|FAC|ASM|ALV|DOTH|DWC|EDU|FOOD|HCF|HIH|HOS|MIXC|INS|MER|LIB|MNTL|MOB|PUB|STO|UT|DFAM') == True]
crehab = crehab[ crehab['cost'] >=5000 ]
crehab = crehab[  crehab['typework'].isin( ['AA','ALT','ADD'] ) ]
crehab.head(1)

In [ ]:
crehab = crehab.groupby('CSA2010').sum(numeric_only=True) 
# Make sure ALL csas and BaltimoreCity are included. among other things
crehab = csa[ ['CSA2010','tpop10'] ].merge( crehab, left_on='CSA2010', right_on='CSA2010', how='outer' ) 
# Update the baltimore CSA.
crehab.at[55,'crehab19'] = crehab['crehab19'].sum() 
crehab = crehab[['crehab19', 'CSA2010']]
crehab['crehab19'] = crehab['crehab19'] *100 / comprop['comprop19']

crehab.head(58)

# Create the Indicator
crehab.to_csv('142-crehab19-19.csv', index=False) 

### Fore 33 - (MdProp, Close Crawl)

In [ ]:
# 33 - fore - percent of properties under mortgage foreclosure

# https://services1.arcgis.com/mVFRs7NF4iFitgbY/arcgis/rest/services/Constper/FeatureServer/layers
# https://bniajfi.org/indicators/Housing%20And%20Community%20Development/fore

# Numerator: foreclosures.foreclosures_201X
# Denominator: housing.mdprop_201X
# run the 2018 and 2019 crawler first!

long_Description: """
The percentage of properties where the lending company or loan servicer has filed a foreclosure proceeding with the 
altimore City Circuit Court out of all residential properties within an area. This is not a measure of actual foreclosures 
since not every property that receives a filing results in a property dispossession.
"""

fore_SQL = """
 2016
  with numerator AS (
   select (sum( case when csa_present then 1 else NULL end)::numeric) as result, csa
    from vital_signs.match_csas_and_bc_by_geom('foreclosures.foreclosures_2016', 'gid', 'the_geom') a
      left join foreclosures.foreclosures_2016 b on a.gid = b.gid 
        group by csa  ),
  denominator AS (
   select (sum( case when (address != $$NULL$$) 
      AND (desclu = $$Apartments$$ 
      OR desclu = $$Residential$$ 
      OR desclu = $$Residential Commercial$$ 
      OR desclu = $$Residential Condominium$$) then 1 else NULL end)::numeric  ) as result, csa
    from vital_signs.match_csas_and_bc_by_geom('housing.mdprop_2017', 'gid', 'the_geom') a
      left join housing.mdprop_2017 b on a.gid = b.gid
        group by csa, the_pop  ),
  tbl AS (
   select denominator.csa,(numerator.result / denominator.result)*(100::numeric) as result 
   from numerator left join denominator on numerator.csa = denominator.csa  )
  update vital_signs.data
  set fore = result from tbl where data.csa = tbl.csa and data_year = '2016'; 

 --/* <fore_16 number> */
  select(sum( case when csa_present then 1 else NULL end)::numeric) as result, csa
   from vital_signs.match_csas_and_bc_by_geom('foreclosures.foreclosures_2016', 'gid', 'the_geom') a
     left join foreclosures.foreclosures_2016 b on a.gid = b.gid
       group by csa order by csa = 'Baltimore City', csa

  WHERE (ooi like 'H') 
  AND (address != $$NULL$$) 
  AND (desclu = $$Apartments$$ 
  OR desclu = $$Residential$$ 
  OR desclu = $$Residential Commercial$$ 
  OR desclu = $$Residential Condominium$$
  """

fore_translation = "( count of closecrawl records per CSA / mdprop_2017.totalres )* 100"

In [ ]:
# Aggregate Numeric Values by Sum 
foreclosuresCsa['foreCount'] = 1
fore = foreclosuresCsa.groupby('CSA2010').sum(numeric_only=True) 
fore = fore.reset_index()
fore = fore.merge( totalres[ ['CSA2010', 'totalres18'] ], left_on='CSA2010', right_on='CSA2010' ) 
fore.tail()

In [ ]:
# Create the Indicator
fore['fore'] = fore['foreCount'] * 100 / fore['totalres18']

fore = fore.append({'CSA2010': 'Baltimore City' , 
                        'foreCount':  fore['foreCount'].sum(), 
                        'fore' : fore['fore'].sum()/55,
                        'totalres18' : 204792}, ignore_index=True)

fore = fore[['CSA2010', 'foreCount', 'fore', 'totalres18' ]]

fore.to_csv('fore.csv', index=False)

fore.tail(60)

### Vacant 34 - (MdProp, Vacants) - DATEABATE, DATECANCLE missing (Is it needed?)

In [ ]:
# 34- vacant - percentage of residential properties that are vacant and abandoned

# https://services1.arcgis.com/mVFRs7NF4iFitgbY/arcgis/rest/services/vacant/FeatureServer/layers 

# Numerator: housing.vacants_201X
# Denominator: housing.mdprop_201X

long_Description: """
The percentage of residential properties that have been classified as being vacant and abandoned by the Baltimore City Department
 of Housing out of all properties. Properties are classified as being vacant and abandoned if: the property is not habitable and
  appears boarded up or open to the elements; the property was designated as being vacant prior to the current year and still
   remains vacant; and the property is a multi-family structure where all units are considered to be vacant.
"""

vacant_SQL = """
 2016
  with numerator AS (
   select (sum( case 
     when (datenotice between '2004-01-01' and '2016-12-31') AND (dateabate is NULL OR dateabate >= '2016-12-31') AND (datecancel is NULL OR datecancel > '2016-12-31') then 1 else NULL end)::numeric) as result, csa
       from vital_signs.match_csas_and_bc_by_geom('housing.vacants_2016', 'gid', 'the_geom') a
         left join housing.vacants_2016 b on a.gid = b.gid
           group by csa ),
  denominator AS (
    select (sum(
     case 
     when (address != $$NULL$$) AND (desclu = $$Apartments$$ OR desclu = $$Residential$$ OR desclu = $$Residential Commercial$$ OR desclu = $$Residential Condominium$$)
     then 1
     else NULL
     end)::numeric 
    ) as result, csa
    from vital_signs.match_csas_and_bc_by_geom('housing.mdprop_2017', 'gid', 'the_geom') a
    left join housing.mdprop_2017 b on a.gid = b.gid
    group by csa, the_pop
   ),
  tbl AS (
     select denominator.csa,(numerator.result / denominator.result)*(100::numeric) as result 
     from numerator left join denominator on numerator.csa = denominator.csa
     )
  update vital_signs.data
  set vacant = result from tbl where data.csa = tbl.csa and data_year = '2016';"

 2017
   with numerator AS (
    select (sum( case 
     when (datenotice between '2004-01-01' and '2017-12-31') AND (dateabate is NULL OR dateabate >= '2017-12-31') AND (datecancel is NULL OR datecancel > '2017-12-31') then 1 else NULL end)::numeric) as result, csa
       from vital_signs.match_csas_and_bc_by_geom('housing.vacants_2017', 'gid', 'the_geom') a
         left join housing.vacants_2017 b on a.gid = b.gid
           group by csa ),
   denominator AS (
    select (sum( case 
     when (address != $$NULL$$) AND (desclu = $$Apartments$$ OR desclu = $$Residential$$ OR desclu = $$Residential Commercial$$ OR desclu = $$Residential Condominium$$)
       then 1 else NULL end)::numeric ) as result, csa
      from vital_signs.match_csas_and_bc_by_geom('housing.mdprop_2017vs', 'gid', 'the_geom') a
        left join housing.mdprop_2017vs b on a.gid = b.gid
          group by csa, the_pop ),
   tbl AS (
     select denominator.csa,(numerator.result / denominator.result)*(100::numeric) as result 
     from numerator left join denominator on numerator.csa = denominator.csa
     )
   select * from tbl order by csa asc"
   column "dateabate" does not exist
  """

vacant_translation = """
Numerator = sum vacants_2017 when 
  (datenotice between '2004-01-01' and '2017-12-31') 
  AND (dateabate is NULL OR dateabate >= '2017-12-31') 
  AND (datecancel is NULL OR datecancel > '2017-12-31')

Denominator =  mdprop_2017.totalres

return = (numerator / denominator )* 100
"""

In [ ]:
cd ../

In [ ]:
cd 'Vacants and Violations'

In [ ]:
ls

In [ ]:
ponp_vacants19 = pd.read_csv('Vacants and Violations/ponp_vacants19.csv')

In [ ]:
vacantsCsa = ponp_vacants19.copy() 
vacantsCsa.head(1)

In [ ]:
vacantsCsa.dtypes

In [ ]:
import datetime

In [ ]:
vacantsCsa['DateNotice2'] = pd.to_datetime(vacantsCsa['DateNotice'],infer_datetime_format=True)

In [ ]:
vacants = vacantsCsa[
    ( vacantsCsa['DateNotice2']>=pd.Timestamp(2005,1,1) ) & 
    ( vacantsCsa['DateNotice2']<=pd.Timestamp(2018,12,31) )
]


vacants.to_csv('vacants_Filtered_Records.csv', index=False)

print( 'Records Matching Query: ', vacants.size / len(vacants.columns) )

# Aggregate Numeric Values by Sum 
vacants['vacantsCount'] = 1
vacants = vacants.groupby('CSA2010').sum(numeric_only=True) 
vacants = vacants.merge( csa[ ['CSA2010'] ], left_on='CSA2010', right_on='CSA2010' ) 
vacants = vacants.merge( totalres[ ['CSA2010', 'totalres18'] ], left_on='CSA2010', right_on='CSA2010' ) 

# Create the Indicator
vacants['vacants'] = vacants['vacantsCount'] * 100 / totalres['totalres18']

# Create Baltimore's Record
vacants = vacants.append({'CSA2010': 'Baltimore City' , 
                            'vacantsCount':  vacants['vacants'].sum(), 
                            'vacants' : vacants['vacants'].sum()/55,
                            'totalres18' : 204792 } , ignore_index=True)

vacants = vacants[ ['CSA2010', 'vacantsCount', 'vacants', 'totalres18' ] ]

vacants.to_csv('vacants.csv', index=False)

vacants.tail(60)

### Vio 35 - (MdProp, Violations) - Complete

In [ ]:
# 35- vio - Percentage of residential properties with housing violations (excluding vacants)

# https://services1.arcgis.com/mVFRs7NF4iFitgbY/arcgis/rest/services/vio/FeatureServer/layers

# Numerator: housing.violations_thru201X
# Denominator: housing.mdprop_201X

long_Description: """
  The percentage of residential properties that have received at least one housing code violation from the Baltimore City 
  Department of Housing out of all properties. Properties whose façade, structure, and/or surrounding area violate the City's 
  Housing Code are issued a notice and are considered open till the property is found in compliance. A property may receive multiple violations.
  """

vio_SQL = """  
 16
  with numerator AS (
   select (sum( case 
    when (datenotice between '2016-01-01' and '2016-12-31') 
    AND (dateabate is NULL OR dateabate >= '2016-12-31') 
    AND (datecancel is NULL OR datecancel > '2016-12-31')  
      then 1 else NULL end)::numeric) as result, csa
      from vital_signs.match_csas_and_bc_by_geom('housing.violations_thru2016', 'gid', 'the_geom') a
        left join housing.violations_thru2016 b on a.gid = b.gid
          group by csa ),
  denominator AS (
    select (sum( case 
      when (address != $$NULL$$) 
        AND (desclu = $$Apartments$$ 
        OR desclu = $$Residential$$
         OR desclu = $$Residential Commercial$$ 
         OR desclu = $$Residential Condominium$$)
        then 1 else NULL end)::numeric ) as result, csa
        from vital_signs.match_csas_and_bc_by_geom('housing.mdprop_2017', 'gid', 'the_geom') a
          left join housing.mdprop_2017 b on a.gid = b.gid
            group by csa, the_pop ),
  tbl AS (
    select denominator.csa,(numerator.result / denominator.result)*(100::numeric) as result 
     from numerator left join denominator on numerator.csa = denominator.csa )
  update vital_signs.data
  set vio = result from tbl where data.csa = tbl.csa and data_year = '2016';
   
 17
  with numerator AS (
   select (sum( case 
     when (datenotice between '2017-01-01' and '2017-12-31') 
       AND (dateabate is NULL OR dateabate >= '2017-12-31') 
       AND (datecancel is NULL OR datecancel > '2017-12-31')  
       then 1 else NULL end)::numeric) as result, csa
       from vital_signs.match_csas_and_bc_by_geom('housing.violations_2017', 'gid', 'the_geom') a
         left join housing.violations_2017 b on a.gid = b.gid
          group by csa ),
  denominator AS (
    select (sum( case 
     when (address != $$NULL$$) 
       AND (desclu = $$Apartments$$ 
         OR desclu = $$Residential$$ 
         OR desclu = $$Residential Commercial$$ 
         OR desclu = $$Residential Condominium$$)
     then 1 else NULL end)::numeric ) as result, csa
       from vital_signs.match_csas_and_bc_by_geom('housing.mdprop_2017v2', 'gid', 'the_geom') a
         left join housing.mdprop_2017v2 b on a.gid = b.gid
           group by csa, the_pop ),
  tbl AS (
     select denominator.csa,(numerator.result / denominator.result)*(100::numeric) as result 
     from numerator left join denominator on numerator.csa = denominator.csa ) 
  select * from tbl where 1 = 1 ORDER BY csa ASC;"

  (datenotice between '2016-01-01' and '2016-12-31') AND (dateabate is NULL OR dateabate >= '2016-12-31') AND (datecancel is NULL OR datecancel > '2016-12-31') 
  """

vio_translation = """
  Numerator = sum violations_2017 when 
    (datenotice between '2018-01-01' and '2018-12-31') AND 
    (dateabate is NULL OR dateabate >= '2018-12-31') AND 
    (datecancel is NULL OR datecancel > '2018-12-31')  
    denominator = mdprop_2017.totalres
    return = (numerator / denominator )* 100
  """

In [ ]:
# https://stackoverflow.com/questions/29314033/drop-rows-containing-empty-cells-from-a-pandas-dataframe
vio = gpd.read_file("Violations_20"+year+"_CSACity.shp", geometry='geometry');
# print(permitsCsa.crs)
# permits = permits.to_crs(epsg=4326)
vio['CSA2010'] = vio['CSA']
vio['bcitycount'] = 1
vio['csacount'] = 1

print( vio[vio['BaltCity'].isnull()].CSA.unique() ) 
vio.head(1)

In [ ]:
# replace empty
vio['BaltCity'].replace('', np.nan, inplace=True)
# drop null
vio.dropna(subset=['BaltCity'], inplace=True)

vio['DateCancel'] = pd.to_datetime(vio['DateCancel'])
vio['DateAbate'] = pd.to_datetime(vio['DateAbate'])
vio['DateNotice'] = pd.to_datetime(vio['DateNotice'], errors='coerce')

# Numerator
vio = vio[['DateNotice', 'DateAbate', 'DateCancel','CSA2010']]
vio.head(1)

start_date = '20'+year+'-01-01'
end_date = '20'+year+'-12-31'

mask = vio[ ( vio['DateNotice'] > start_date ) & ( vio['DateNotice'] <= end_date) ]
mask1 = mask[ ( pd.isnull( mask['DateAbate'] ) ) | ( mask['DateAbate'] >= end_date ) ]
mask2 = mask1[ pd.isnull( mask1['DateCancel'] ) | ( mask1['DateCancel'] > end_date ) ]
vio2 = mask2.copy()

vio2.to_csv('vio_Filtered_Records.csv', index=False)

In [ ]:
print( 'Records Matching Query: ', vio.shape[0] )

# Aggregate Numeric Values by Sum 
vio['vioCount'] = 1
vio = vio.groupby('CSA2010').sum(numeric_only=True) 
vio = vio.reset_index()

# Remove the 'False' Records
reapp = vio.iloc[-1:]
vio = vio.drop( vio.index[-1:] )

# Create the Indicator 
vio['vio'+year] = vio['vioCount'] * 100 / vioDenominator['totalres'+year] 

# Lets see what we got
vio.head(2)
vio.tail(2)
vio.shape[0]

In [ ]:
# Denominator
vioDenominator = totalres.copy()
vioDenominator = vioDenominator[['totalres'+year,'CSA2010']]

vioDenominator.head(2)
vioDenominator.shape[0]

# Create Baltimore's Record
# Ownroc - Baltimore City	54.6
vio = vio.append({'CSA2010': 'Baltimore City' , 
                            'vioCount':  vio['vio'+year].sum(), 
                            'vio' : vio['vio'+year].sum()/54.6 } , ignore_index=True)
# Reappend the False records
vio = vio.append(reapp)

# Ensure all CSA's are listed
vio = pd.merge(vio,csa,on='CSA2010',how='outer')
vio = vio[['CSA2010', 'vioCount', 'vio']]

# Save
vio.to_csv('vio.csv', index=False)

vio.head()
vio.tail()

In [ ]:
vio[ vio['DateNotice']=='Pigtown Phase 9']

In [ ]:
vio.head(1)

### Resrehab 36 - (MdProp, Permits) - Complete

In [ ]:
# 36- resrehab - Percentage of properties with rehabilitation permits exceeding $5k

# https://services1.arcgis.com/mVFRs7NF4iFitgbY/arcgis/rest/services/resrehab/FeatureServer/layers

# Numerator: housing.permits_201X
# Denominator: housing.mdprop_201X

long_Description: """
  The percent of residential properties that have applied for and received a permit to renovate the interior and/or exterior 
  of a property where the cost of renovation will exceed $5,000. The threshold of $5,000 is used to differentiate a minor 
  and more significant renovation project.
  """

resrehab_SQL = """
  2016
    with numerator AS (
    select sum( case 
      when (exis = $$SF$$ OR exis = $$MF$$ ) AND (type1 = $$AA$$ OR type1 = $$ADD$$ OR type1 = $$ALT$$) AND (costts >=5000)
        then 1 else 0 end)::numeric as result, csa from vital_signs.match_csas_and_bc_by_geom('housing.permits_2016', 'gid', 'the_geom') a
        left join housing.permits_2016 b on a.gid = b.gid
          group by csa ),
    denominator AS (
      select (sum( case 
      when (address != $$NULL$$) AND (desclu = $$Apartments$$ OR desclu = $$Residential$$ OR desclu = $$Residential Commercial$$ OR desclu = $$Residential Condominium$$)
        then 1 else NULL end)::numeric ) as result, csa
          from vital_signs.match_csas_and_bc_by_geom('housing.mdprop_2017', 'gid', 'the_geom') a
            left join housing.mdprop_2017 b on a.gid = b.gid
              group by csa, the_pop ),
    tbl AS (
      select denominator.csa,(numerator.result / denominator.result)*(100::numeric) as result 
      from numerator left join denominator on numerator.csa = denominator.csa )
    update vital_signs.data
    set resrehab = result from tbl where data.csa = tbl.csa and data_year = '2016';"

  2017
    with numerator AS (
    select sum(case 
      when (existingus = $$SF$$ 
              OR propuse = $$SF$$ 
              OR existingus = $$MF$$ 
              OR propuse = $$MF$$ 
              OR existingus = $$DFAM$$ 
              OR propuse = $$DFAM$$ 
              OR existingus like '%1-%'
              OR propuse like '%1-%' 
            ) 
            AND casetype LIKE any (ARRAY['COM'])
            AND (field22 = $$AA$$ OR field22 = $$ADD$$ OR field22 = $$ALT$$) 
            AND (cost >=5000) then 1 else 0 end )::numeric as result, csa
      from vital_signs.match_csas_and_bc_by_geom('housing.permits_2017', 'gid', 'the_geom') a
        left join housing.permits_2017 b on a.gid = b.gid
          group by csa ),
    denominator AS (
    select (sum( case 
        when (address != $$NULL$$) AND (desclu = $$Apartments$$ OR desclu = $$Residential$$ OR desclu = $$Residential Commercial$$ OR desclu = $$Residential Condominium$$) then 1 else NULL end )::numeric ) as result, csa
      from vital_signs.match_csas_and_bc_by_geom('housing.mdprop_2017v2', 'gid', 'the_geom') a
        left join housing.mdprop_2017v2 b on a.gid = b.gid
          group by csa, the_pop ),
    tbl AS (
    select denominator.csa,(numerator.result / denominator.result)*(100::numeric) as result 
      from numerator left join denominator on numerator.csa = denominator.csa )
    select * from tbl where 1 = 1 ORDER BY csa ASC;"

    INVALID COLUMN NAMES
    "/* FIELD22: NEW ALT OTH DEM AA ADD */
    2016 - > exis, prop, type1, cossts
    2017 -> existingus, propuse, field22, cost
    """

resrehab_translation = """
  Numerator = sum permits_2017 when (
              existingus = $$SF$$ OR existingus = $$MF$$ OR existingus = $$DFAM$$ OR existingus like '%1-%'
              OR propuse = $$SF$$ OR propuse = $$MF$$ OR propuse = $$DFAM$$ OR propuse like '%1-%' 
            ) 
            AND casetype LIKE any (ARRAY['COM'])
            AND (field22 = $$AA$$ OR field22 = $$ADD$$ OR field22 = $$ALT$$) 
            AND (cost >=5000) then 1 else 0 end )
  Denominator = mdprop.totalres
  return = (numerator / denominator )* 100
  """

In [ ]:
year
totalres.head()

In [ ]:
resrehab = permitsCsa
resrehab['Field22'] = resrehab['typework']

use = ".SF.|.MF.|.DFAM.|.1-.|SF|MF|DFAM|1-.|.1-" 
resrehab = resrehab[ 
  ( permitsCsa['existingus'].str.contains(use, regex=True, na=False) ) & 
  ( permitsCsa['propuse'].str.contains(use, regex=True, na=False) ) & 
  ( permitsCsa['casetype'].str.contains('.COM.|COM', regex=True, na=False) ) & 
  ( permitsCsa['Field22'].str.contains('.AA.|.ADD.|.ALT.|AA|ADD|ALT|ADD', regex=True, na=False) ) & 
  ( permitsCsa['cost'] >=5000 ) 
] 

resrehab.to_csv('resrehab'+year+'_Filtered_Records.csv', index=False)

print( 'Records Matching Query: ', resrehab.size / len(resrehab.columns) )

# Aggregate Numeric Values by Sum 
resrehab['resrehabCount'] = 1
resrehab = resrehab.groupby('CSA2010').sum(numeric_only=True) 

# Make sure ALL csas and BaltimoreCity are included. among other things
resrehab = totalres[ ['CSA2010','totalres'+year] ].merge( resrehab, left_on='CSA2010', right_on='CSA2010', how='outer' ) 

# Update the baltimore CSA.
resrehab.at[55,'resrehabCount'] = resrehab['resrehabCount'].sum() 

# Create the Indicator
resrehab['resrehab'+year] = resrehab['resrehabCount'] * 100 / totalres['totalres'+year]

resrehab = resrehab[ ['CSA2010', 'resrehabCount', 'resrehab'+year, 'totalres'+year ] ]

resrehab.to_csv('resrehab'+year+'.csv', index=False)

resrehab.head()
resrehab.tail()

### 37 - Total Res

In [ ]:
mdprop = gpd.read_file('MDPropertyView_2021_CSACity.shp')
mdprop.head()

In [ ]:
mdprop = mdprop.CSA.fillna('Baltimore City')
mdprop['totalres19'] = 1
mdprop = mdprop[['CSA','totalres20']]
mdprop = mdprop.groupby('CSA').sum(numeric_only=True) 
# Make sure ALL csas and BaltimoreCity are included. among other things
mdprop = csa[ ['CSA2010','tpop10'] ].merge( mdprop, left_on='CSA2010', right_on='CSA', how='outer' ) 
# Update the baltimore CSA.
mdprop.at[55,'totalres20'] = mdprop['totalres20'].sum() 
mdprop.head(58)

In [ ]:
# Create the Indicator
mdprop.to_csv('37-totalres-19.csv', index=False) 

In [ ]:
mdprop.CSA.unique()

### Demper 41 - (MdProp, Permits) - Complete

In [ ]:
# 41- demper - number of demolition permits per 1000 residential properties

# https://services1.arcgis.com/mVFRs7NF4iFitgbY/arcgis/rest/services/demper/FeatureServer/layers

# Numerator: housing.dempermits_201X
# Denominator: housing.mdprop_201X

long_Description: """
  The number of permits issued for the demolition of residential buildings per 1,000 existing residential properties. 
  The permits are analyzed by date of issue and not date of actual demolition.
  """

demper_SQL = """
  2016
    with numerator AS (
    select (sum( case 
      when csa_present then 1 else 0 end)::numeric) as result, csa
        from vital_signs.match_csas_and_bc_by_geom('housing.dempermits_2016', 'gid', 'the_geom') a
          left join housing.dempermits_2016 b on a.gid = b.gid
            group by csa ),
    denominator AS (
      select (sum(
      case 
      when (address != $$NULL$$) AND (desclu = $$Apartments$$ OR desclu = $$Residential$$ OR desclu = $$Residential Commercial$$ OR desclu = $$Residential Condominium$$)
      then 1
      else NULL
      end)::numeric 
      ) as result, csa
      from vital_signs.match_csas_and_bc_by_geom('housing.mdprop_2017', 'gid', 'the_geom') a
      left join housing.mdprop_2017 b on a.gid = b.gid
      group by csa, the_pop ),
    tbl AS (
      select denominator.csa,(numerator.result / denominator.result)*(1000::numeric) as result 
      from numerator left join denominator on numerator.csa = denominator.csa )
    update vital_signs.data
    set demper = result from tbl where data.csa = tbl.csa and data_year = '2016'; "
  
  2017
    with numerator AS (
    select (sum( 
      case 
        when csa_present AND casetype LIKE any ( ARRAY['DEM'] ) AND planaddres != '' 
          then 1 else 0
            end
    )::numeric) as result, csa
      from vital_signs.match_csas_and_bc_by_geom('housing.permits_2017', 'gid', 'the_geom') a
        left join housing.permits_2017 b on a.gid = b.gid
          group by csa ),
    denominator AS (
    select (sum( case 
      when (address != $$NULL$$) AND 
          (desclu = $$Apartments$$ 
            OR desclu = $$Residential$$ 
            OR desclu = $$Residential Commercial$$ 
            OR desclu = $$Residential Condominium$$ 
      ) then 1 else NULL end )::numeric ) as result, csa
        from vital_signs.match_csas_and_bc_by_geom('housing.mdprop_2017v2', 'gid', 'the_geom') a
          left join housing.mdprop_2017v2 b on a.gid = b.gid
            group by csa, the_pop ),
    tbl AS (
    select denominator.csa,(numerator.result / denominator.result)*(1000::numeric) as result 
      from numerator left join denominator on numerator.csa = denominator.csa 
    select * from tbl where 1 = 1 ORDER BY csa ASC;"

  INVALID COLUMN NAMES
  2016 - > exis, prop, type1, cossts
  2017 -> existingus, propuse, field22, cost
  """

demper_translation = """ ( sum permits_2017 when csa_present AND casetype LIKE any ( ARRAY['DEM'] ) AND planaddres != '' / mdprop.totalres )* 1000 """

In [ ]:
demper = permitsCsa[ 
  ( permitsCsa['casetype'].str.contains('DEM|.DEM.|DEM.|.DEM', regex=True, na=False) ) 
]

filter = demper["PLANADDRES"] != ""
demper = demper[filter]


demper.to_csv('demper'+year+'_Filtered_Records.csv', index=False)

print( 'Records Matching Query: ', demper.size / len(demper.columns) )


# Aggregate Numeric Values by Sum 
demper['demperCount'] = 1
demper = demper.groupby('CSA2010').sum(numeric_only=True) 

# Make sure ALL csas and BaltimoreCity are included. among other things
demper = totalres[ ['CSA2010','totalres'+year] ].merge( demper, left_on='CSA2010', right_on='CSA2010', how='outer' ) 

# Update the baltimore CSA.
demper.at[55,'demperCount'] = demper['demperCount'].sum() 

# Create the Indicator
demper['demper'+year] = demper['demperCount'] * 100 / totalres['totalres'+year]

# Create the Indicator
demper['demper'+year] = demper['demperCount'] * 1000 / totalres['totalres'+year]

demper = demper[['CSA2010', 'demperCount', 'demper'+year, 'totalres'+year ]]

demper.to_csv('demper'+year+'.csv', index=False)

demper.head()
demper.tail()

### Constper 42 - (MdProp, Permits) - Complete

In [ ]:
# 42- constper - Number of of new constructio permits per 1000 residential properties

# https://services1.arcgis.com/mVFRs7NF4iFitgbY/arcgis/rest/services/constper/FeatureServer/layers

# Numerator: housing.permits_201X
# Denominator: housing.mdprop_201X

long_Description: """
  The number of permits issued for new residential buildings per 1,000 existing residential properties within a community. 
  The permits are analyzed by date of issue and not date of completion.
  """

constper_SQL = """
 2016
  with numerator as (
   select sum( case 
    when (prop = $$SF$$ OR prop = $$MF$$) AND (type1 = $$NEW$$) then 1 else 0 end)::numeric as result, csa
      from vital_signs.match_csas_and_bc_by_geom('housing.permits_2016', 'gid', 'the_geom') a
        left join housing.permits_2016 b on a.gid = b.gid
          group by csa ),
  denominator AS (
   select (sum( case 
    when (address != $$NULL$$) AND (desclu = $$Apartments$$ OR desclu = $$Residential$$ OR desclu = $$Residential Commercial$$ OR desclu = $$Residential Condominium$$)
      then 1 else NULL end)::numeric ) as result, csa
        from vital_signs.match_csas_and_bc_by_geom('housing.mdprop_2017', 'gid', 'the_geom') a
          left join housing.mdprop_2017 b on a.gid = b.gid
            group by csa, the_pop ),
  tbl AS (
   select denominator.csa,(numerator.result / denominator.result)*(1000::numeric) as result 
   from numerator left join denominator on numerator.csa = denominator.csa )
  update vital_signs.data
  set constper = result from tbl where data.csa = tbl.csa and data_year = '2016'; "

 2017
  with numerator as (
   select sum( 
    case 
      when (existingus = $$SF$$
      OR propuse = $$SF$$
      OR existingus = $$MF$$
      OR propuse = $$MF$$
      OR existingus = $$DFAM$$
      OR propuse = $$DFAM$$
      OR existingus like '%1-%'
      OR propuse like '%1-%'
    ) AND (field22 = $$NEW$$)
      AND casetype LIKE any ( ARRAY['COM'] ) 
        then 1 else 0 end )::numeric as result, csa
     from vital_signs.match_csas_and_bc_by_geom('housing.permits_2017', 'gid', 'the_geom') a
       left join housing.permits_2017 b on a.gid = b.gid
         group by csa ),
  denominator AS (
    select (sum( case 
      when (address != $$NULL$$) 
         AND (desclu = $$Apartments$$ 
            OR desclu = $$Residential$$ 
            OR desclu = $$Residential Commercial$$ 
            OR desclu = $$Residential Condominium$$
         ) then 1 else NULL end )::numeric ) as result, csa
   from vital_signs.match_csas_and_bc_by_geom('housing.mdprop_2017v2', 'gid', 'the_geom') a
     left join housing.mdprop_2017v2 b on a.gid = b.gid
       group by csa, the_pop ),
  tbl AS (
    select denominator.csa,(numerator.result / denominator.result)*(1000::numeric) as result 
    from numerator left join denominator on numerator.csa = denominator.csa )
  select * from tbl where 1 = 1 ORDER BY csa ASC;"

  INVALID COLUMN NAMES
  2016 - > exis, prop, type1, cossts
  2017 -> existingus, propuse, field22, cost
  """

constper_translation = """
  Numerator = sum permits_2017 when (existingus = $$SF$$ 
    OR existingus = $$DFAM$$ OR existingus = $$MF$$ OR existingus like '%1-%'
    OR propuse = $$SF$$ OR propuse = $$MF$$ OR propuse = $$DFAM$$ OR propuse like '%1-%'
    ) 
    AND (field22 = $$NEW$$)
    AND casetype LIKE any ( ARRAY['COM'] ) 
  Denominator = mdprop.totalres

  return = (numerator / Denominator )* 1000
  """

In [ ]:
# 2018 and 2017 is not working with the new datasets given (CSA LABELS)

In [ ]:
permitsCsa.head()

In [ ]:
permitsCsa.casetype.unique()

In [ ]:
use = "SF|MF|.SF.|.MF.|.SF|.MF|SF.|MF."
constper = permitsCsa
#constper['Field22'] = constper['typework']
constper = constper[ 
  #( 
  #    permitsCsa['existingus'].str.contains(use, regex=True, na=False) |
  #    permitsCsa['propuse'].str.contains(use, regex=True, na=False)
  #) & 
  #( permitsCsa['casetype'].str.contains('COM|.COM.|COM.|.COM', regex=True, na=False) ) &  
  ( permitsCsa['Field22'].str.contains('NEW|.NEW.|NEW.|.NEW', regex=True, na=False) )
]

filter = constper["PLANADDRES"] != ""
constper = constper[filter]

constper = constper[['CSA2010','existingus','propuse','casetype','Field22','PLANADDRES' ]]

constper.to_csv('constper'+year+'_Filtered_Records.csv', index=False)

print( 'Records Matching Query: ', constper.size / len(constper.columns) )

# Aggregate Numeric Values by Sum 
constper['constperCount'] = 1
constper = constper.groupby('CSA2010').sum(numeric_only=True) 

# Make sure ALL csas and BaltimoreCity are included. among other things
constper = totalres[ ['CSA2010','totalres'+year] ].merge( constper, left_on='CSA2010', right_on='CSA2010', how='outer' ) 

# Update the baltimore CSA.
constper.at[55,'constperCount'] = constper['constperCount'].sum() 

# Create the Indicator
constper['constper'+year] = constper['constperCount'] * 1000 / totalres['totalres'+year]

constper.to_csv('constper'+year+'.csv', index=False)

constper.head()
constper.tail()